<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN03/3.1-Klassifizierung_News-PyTorch.ipynb)

# Ein Beispiel für eine Mehrfachklassifizierung:

Dieses Notebook enthält die Codebeispiele aus Kapitel 3, Abschnitt 5 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).


## Klassifizierung von Nachrichtenmeldungen

Im letzten Abschnitt haben Sie erfahren, wie man mit vollständig verbundenen NNs Vektoreingaben klassifiziert und sie zwei sich gegenseitig ausschliessenden Klassen zuordnet. Aber was ist, wenn es mehr als zwei Klassen gibt?

- In dieser Lektion werden wir ein NN entwickeln, das Nachrichtenmeldungen der Agentur Reuters 46 sich gegenseitig ausschliessenden Themenbereichen zuordnet. Da es hier mehrere Klassen gibt, spricht man von einer *Mehrfachklassifizierung*.
- Und weil jeder Datenpunkt nur einer einzigen Kategorie zugeordnet werden darf, handelt es sich genau genommen um eine *Single-Label-Mehrfachklassifizierung.*
- Wenn die Datenpunkte mehreren Kategorien (in diesem Fall Themengebieten) angehören dürfen, haben Sie es hingegen mit einer Multi-Label-Mehrfachklassifizierung zu tun.

## Die Reuters-Datensammlung

Wir werden die Reuters-Datensammlung verwenden, eine aus kurzen Nachrichtenmeldungen und ihren Themengebieten bestehende Datenmenge, die 1986 von Reuters veröffentlicht wurde. Diese einfache Datenmenge wird häufig für Textklassifizierungsexperimente genutzt. Es gibt 46 verschiedene Themengebiete. Einige davon enthalten mehr Meldungen als andere, aber in der Trainingsmenge liegen zu jedem Themengebiet mindestens zehn Meldungen vor.


Ebenso wie die IMDb- und die MNIST-Datensammlung ist auch die Reuters-Datensammlung Bestandteil von Keras. Sehen wir uns das genauer an.

In [ ]:
num_words = 10000

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt


In [ ]:
# Dateien laden
!wget -O reuters_dataset_with_word_index.pkl "https://github.com/ChristophWuersch/AppliedNeuralNetworks/raw/refs/heads/main/ANN02/Daten/reuters_dataset_with_word_index.pkl"

In [ ]:
# Imports
import sys
import torch
import pickle
import matplotlib
import numpy as np
import pandas as pd
import torch.nn as nn
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from torchviz import make_dot
from torchsummary import summary

from IPython.display import Image, display
from torch.utils.data import Dataset, DataLoader

# Save the dataset and word index into a pickle file
pickle_file_path = "reuters_dataset_with_word_index.pkl"

# Reload the data from the pickle file
with open(pickle_file_path, "rb") as file:
    ibdm_data = pickle.load(file)

# Verify reloaded word index matches the original
word_index = ibdm_data["word_index"]
list(word_index.items())[:5]

In [ ]:
train_data = ibdm_data["train_data"]
test_data = ibdm_data["test_data"]

train_labels = ibdm_data["train_labels"]
test_labels = ibdm_data["test_labels"]


- Wie bei der IMDb-Datensammlung beschränkt das Argument `num=10000` die Daten auf die 10.000 am häufigsten vorkommenden Wörter.
- Es gibt 8.982 Trainings- und 2.246 Testdatensätze:

In [ ]:
len(train_data)

In [ ]:
len(test_data)

Und wie die Bewertungen der IMDb-Datensammlung bestehen die Datensätze aus einer Liste von Integern (Wortindizes):

In [ ]:
train_data[10]

Falls Sie möchten, können Sie die Daten wieder in Wörter umwandeln:

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])

In [ ]:
decoded_newswire

Den Datensätzen ist ein Integer zwischen 0 und 45 als Themenindex zugeordnet:

In [ ]:
train_labels[10]

## Daten vorbereiten (data preprocessing)
Die *Vektorisierung der Daten* (tidy dataset) können Sie mit genau demselben Code wie beim letzten Beispiel vornehmen.

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results


# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)


Für die Vektorisierung der Klassenbezeichnungen (Labels) gibt es zwei Möglichkeiten:

1. Sie können die Liste entweder in einen **Integertensor** umwandeln oder eine **One-hot-Codierung** verwenden. 
- Die **One-hot-Codierung** ist ein für kategoriale Daten gebräuchliches Format und wird mitunter auch als kategoriale Codierung bezeichnet. 
- Im vorliegenden Fall wird für die One-hot-Codierung der Klassenbezeichnungen ein aus Nullen bestehender Vektor verwendet, der an der Indexposition der Klassenbezeichnung eine 1 enthält. Hier ein Beispiel:

In [ ]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.0
    return results


# Our vectorized training labels
one_hot_train_labels = to_one_hot(train_labels)
# Our vectorized test labels
one_hot_test_labels = to_one_hot(test_labels)

In [ ]:
one_hot_train_labels[:, 11].T

Beachten Sie, dass Keras hierfür eine integrierte Möglichkeit bietet, die Sie beim MNIST-Beispiel auch schon in Aktion gesehen haben:

In [ ]:
# Print versions in a compact form
print(f"Python version: {sys.version}")
print(f"torch: {torch.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")

In [ ]:
# Check for GPU
import torch

torch.cuda.is_available()

In [ ]:
# Set device type
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class ReutersDataset(Dataset):
    """
    A PyTorch Dataset class for the Reuters dataset.
    """

    def __init__(self, data, labels):
        """
        Args:
            data (numpy.ndarray): The feature data (one-hot encoded).
            labels (numpy.ndarray): The labels (one-hot encoded).
        """
        self.data = torch.tensor(data, dtype=torch.float32).to(device)
        self.labels = torch.tensor(labels, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Create dataset instances
train_dataset = ReutersDataset(x_train, one_hot_train_labels)
test_dataset = ReutersDataset(x_test, one_hot_test_labels)

# Create DataLoader instances for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Example usage
for batch_idx, (data, labels) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}")
    print(f"Data shape: {data.shape}")
    print(f"Labels shape: {labels.shape}")
    break


## Bauen wir unser Neuronales Netz auf


- Auf den ersten Blick ähnelt die Klassifizierung nach Themengebieten der Klassifizierung der Filmbewertungen, denn in beiden Fällen versuchen wir, kurze Textabschnitte zu klassifizieren. 
- Der Unterschied besteht hier jedoch darin, dass die Anzahl der möglichen Klassenbezeichnungen von 2 auf 46 gestiegen ist. **Die Dimensionalität des Ausgaberaums ist sehr viel grösser**.
- Bei einem Stapel von Dense-Layern, wie wir ihn bislang verwendet haben, kann jeder Layer nur auf die in der Ausgabe des vorhergehenden Layers enthaltenen Informationen zugreifen. Wenn in einem Layer für die Lösung der Klassifizierungsaufgabe relevante Informationen verloren gehen, können diese in den nachfolgenden Layern nicht wiederhergestellt werden: Jeder Layer kann potenziell zu einem Informationsleck werden. 

Im letzten Beispiel haben Sie 16-dimensionale zwischenliegende Layer verwendet, aber ein 16-dimensionaler Raum ist womöglich
nicht ausreichend, um die Unterscheidung von 46 verschiedenen Klassen zu erlernen: Die möglicherweise zu kleinen Layer könnten zu einem **Informationsleck** werden und relevante Informationen dauerhaft entfernen.
Aus diesem Grund sollten Sie grössere Layer verwenden. 

Versuchen wir es mit 64 Einheiten:

In [ ]:
class ReutersLightningModel(pl.LightningModule):
    """
    A PyTorch Lightning implementation of the Keras Sequential model.
    """

    def __init__(self):
        super(ReutersLightningModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(10000, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 46),
            nn.Softmax(dim=1),
        )
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", accuracy, on_epoch=True, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(accuracy.item())
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("val_accuracy", accuracy, on_epoch=True, prog_bar=True, sync_dist=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(accuracy.item())
        return {"val_loss": loss, "val_accuracy": accuracy}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [ ]:
# DataLoader setup
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model instantiation
model = ReutersLightningModel().to(device)


In [ ]:
# Example input size: (batch_size, input_features)
summary(model, input_size=(batch_size, 10000))

In [ ]:
# Example input tensor
example_input = torch.randn(1, 10000).to(device)

# Get the model graph
graph = make_dot(model(example_input), params=dict(model.named_parameters()))

# Save the graph to a file
graph.render("model_visualization", format="png", cleanup=True)

# Display the visualization in Jupyter Notebook
display(Image("model_visualization.png"))

In [ ]:
# Trainer setup
max_epochs = 10
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1)
trainer.fit(model, train_loader, validation_loader)


Bei dieser Architektur sind zwei weitere Dinge zu beachten:

- Das NN endet mit einem **Dense-Layer der Größe 46**. Das heisst, dass das NN für jede Eingabe einen 46-dimensionalen Vektor ausgibt. Jedes Element (jede Dimension) dieses Vektors codiert eine andere Klassenbezeichnung.
- Der letzte Layer verwendet eine **softmax-Aktivierungsfunktion**. Sie kennen diese Vorgehensweise bereits vom MNIST-Beispiel. - Die Ausgabe des NNs ist eine **Wahrscheinlichkeitsverteilung** der 46 verschiedenen Klassenbezeichnungen – das NN erzeugt für jede Eingabe einen 46-dimensionalen Ausgabevektor, wobei `output[i]` die Wahrscheinlichkeit dafür angibt, dass das Sample zur Klasse `i` gehört. Die Summe der 46 Scores beträgt 1.

In diesem Fall ist die **kategoriale Kreuzentropie (categorial_crossentropy)** die am besten geeignete Verlustfunktion. Sie *misst die Differenz zwischen zwei Wahrscheinlichkeitsverteilungen* – hier die Differenz zwischen der vom NN ausgegebenen
Wahrscheinlichkeitsverteilung und der tatsächlichen Verteilung der Klassenbezeichnungen. Durch die Minimierung der Differenz zwischen diesen beiden Verteilungen wird das NN darauf trainiert, eine Ausgabe zu erzeugen, die der tatsächlichen Verteilung der Klassen so nahe wie möglich kommt.

In [ ]:
metrics = trainer.callback_metrics
metrics.keys()
train_loss = metrics["train_loss_epoch"].cpu().numpy()
train_loss

In [ ]:
# Function to plot learning curves
def plot_learning_curves(model):
    epochs_train = (
        np.array(range(1, len(model.train_losses) + 1))
        / len(model.train_losses)
        * max_epochs
    )
    epochs_val = (
        np.array(range(1, len(model.val_losses) + 1))
        / len(model.val_losses)
        * max_epochs
    )

    # Plotting
    plt.figure(figsize=(12, 5))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_train, model.train_losses, "b.-", label="Training Loss")
    plt.plot(epochs_val, model.val_losses, "r.-", label="Validation Loss")
    plt.title("Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_train, model.train_accuracies, "b.-", label="Training Accuracy")
    plt.plot(epochs_val, model.val_accuracies, "r.-", label="Validation Accuracy")
    plt.title("Accuracy Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot learning curves
plot_learning_curves(model)

Nach neun Epochen kommt es zu einer **Überanpassung**. Jetzt trainieren wir wieder
ein völlig neues NN neun Epochen lang und beurteilen es anschliessend anhand
der Testdaten.


Dieser Ansatz erzielt eine Korrektklassifizierungsrate von knapp 80%. Eine vollkommen auf Zufall beruhende Binärklassifizierung einer Datenmenge mit ausgewogener Verteilung würde eine Korrektklassifizierungsrate von 50% erreichen.

Im vorliegenden Fall einer nicht ausgewogen verteilten Datenmenge läge der Wert bei etwa 19%. So gesehen ist das Ergebnis ziemlich gut, zumindest im Vergleich mit einer rein zufälligen Klassifizierung:

In [ ]:
# Funktion, um Vorhersagen für zufällige Samples auszugeben und zu visualisieren
def plot_random_predictions(model, test_data, num_samples=5):
    model.eval()  # Setze das Modell in den Evaluierungsmodus
    random_indices = np.random.choice(len(test_data), size=num_samples, replace=False)
    samples = [test_data[i] for i in random_indices]

    plt.figure(figsize=(15, 3))

    for i, sample in enumerate(samples):
        input_data = torch.tensor(sample, dtype=torch.float32).unsqueeze(
            0
        )  # Batch-Dimension hinzufügen
        with torch.no_grad():
            predictions_batch = model(input_data)  # Vorhersage berechnen
            predictions = (
                predictions_batch.squeeze().numpy()
            )  # Batch-Dimension entfernen

        # Barplot der Softmax-Wahrscheinlichkeiten
        plt.subplot(1, num_samples, i + 1)
        plt.bar(range(len(predictions)), predictions)
        plt.title(f"Sample {random_indices[i]}")
        plt.xlabel("Klassen")
        plt.grid(True)
        plt.ylabel("Wahrscheinlichkeit")

    plt.tight_layout()
    plt.show()
    return predictions


# Beispielaufruf der Funktion
predictions = plot_random_predictions(model, x_test)


In [ ]:
predictions.sum()

Das Element mit dem größten Wert gibt die vorhersagte Klasse an – die Klasse mit
der höchsten Wahrscheinlichkeit:

In [ ]:
yhat = np.argmax(predictions)
yhat

## Eine weitere Möglichkeit zur Handhabung der Klassenbezeichnungen und der Verlustfunktion

Wie bereits kurz erwähnt, gibt es auch die Möglichkeit, die Klassenbezeichnungen
wie folgt in einen Integertensor umzuwandeln:

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)


Das Einzige, was sich bei diesem Ansatz ändert, ist die Wahl der Verlustfunktion.
- Die oben eingesetzte Verlustfunktion (`categorial_crossentropy`) erwartet, dass die Klassenbezeichnungen *kategorial codiert* sind. 
- Wenn die Klassenbezeichnungen Integer sind, sollten Sie `sparse_categorical_crossentropy`verwenden:

Mathematisch betrachtet, liefert diese Verlustfunktion das gleiche Ergebnis wie
`categorial_crossentropy`, sie besitzt lediglich eine andere Schnittstelle.

## Hinreichend große zwischenliegende Layer sind wichtig

Weil die endgültige Ausgabe, wie bereits kurz erwähnt, 46-dimensional ist, sollten Sie vermeiden, zwischenliegende Layer mit deutlich weniger als 46 Einheiten zu verwenden. Sehen wir uns doch einmal an, was geschieht, wenn es ein Informationsleck
in Form eines zwischenliegenden Layers mit beträchtlich weniger als 46 Dimensionen gibt, beispielsweise eine vierdimensionale.

In [ ]:
class ReutersLightningModel_tight(pl.LightningModule):
    """
    A PyTorch Lightning implementation of the Keras Sequential model.
    """

    def __init__(self):
        super(ReutersLightningModel_tight, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(10000, 64),
            nn.ReLU(),
            nn.Linear(64, 4),
            nn.ReLU(),
            nn.Linear(4, 46),
            nn.Softmax(dim=1),
        )
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", accuracy, on_epoch=True, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(accuracy.item())
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("val_accuracy", accuracy, on_epoch=True, prog_bar=True, sync_dist=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(accuracy.item())
        return {"val_loss": loss, "val_accuracy": accuracy}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [ ]:
# Model instantiation
model2 = ReutersLightningModel_tight()

In [ ]:
# Trainer setup
max_epochs = 10
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1)
trainer.fit(model2, train_loader, validation_loader)


Das NN erreicht bei der Validierung eine Korrektklassifizierungsrate von rund 71%, das ist eine Verschlechterung von 8%. Diese Abnahme ist hauptsächlich der Tatsache geschuldet, dass Sie hier versuchen, eine Menge Informationen (die ausreichen,
um die trennenden Hyperebenen von 46 Klassen zu berechnen) in einen zwischenliegenden Raum einzupferchen, der nicht genügend Dimensionen besitzt. Das NN kann zwar die meisten erforderlichen Informationen in diese achtdimensionalen Repräsentationen hineinstopfen, aber eben nicht alle.

In [ ]:
# Plot learning curves
plot_learning_curves(model2)

## Regularisierung mit Dropout-Layern

In [ ]:
class ReutersLightningModel_dropout(pl.LightningModule):
    """
    A PyTorch Lightning ReutersLightningModel_dropout of the Keras Sequential model.
    """

    def __init__(self):
        super(ReutersLightningModel_dropout, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(10000, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 46),
            nn.Softmax(dim=1),
        )
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", accuracy, on_epoch=True, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(accuracy.item())
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("val_accuracy", accuracy, on_epoch=True, prog_bar=True, sync_dist=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(accuracy.item())
        return {"val_loss": loss, "val_accuracy": accuracy}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [ ]:
# Model instantiation
model3 = ReutersLightningModel_dropout()

In [ ]:
# Trainer setup
max_epochs = 10
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1)
trainer.fit(model3, train_loader, validation_loader)

In [ ]:
# Plot learning curves
plot_learning_curves(model3)

## Gewichtregulierung (Weight Regularization) und die vorgenommenen Änderungen

Die Gewichtregulierung (Weight Regularization) ist eine Technik in neuronalen Netzwerken, die verwendet wird, um das Modell vor Überanpassung (Overfitting) zu schützen. Dabei wird ein Strafterm zu der Verlustfunktion hinzugefügt, der die Größe der Gewichte begrenzt. Ziel ist es, die Komplexität des Modells zu kontrollieren und die Generalisierungsfähigkeit auf unbekannte Daten zu verbessern.

Die **L2-Regularisierung**, auch als **Ridge Regularization** bekannt, minimiert die Summe der quadrierten Werte der Gewichte. Der Strafterm wird wie folgt definiert:

$$
R_{\text{L2}} = \lambda \sum_{i} w_i^2
$$

Hierbei ist:
- $ w_i $: Das Gewicht eines Parameters im Modell
- $ \lambda $: Der Regularisierungsparameter, der das Gewicht des Strafterms bestimmt.

Die Gesamtverlustfunktion wird durch die L2-Regularisierung wie folgt erweitert:

$$
\mathcal{L}_{\text{gesamt}} = \mathcal{L}_{\text{original}} + \lambda \sum_{i} w_i^2
$$

Dabei ist $ \mathcal{L}_{\text{original}} $ die ursprüngliche Verlustfunktion. Bei Klassifikationsproblemen mit Kreuzentropieverlust sieht die erweiterte Verlustfunktion folgendermaßen aus:

$$
\mathcal{L}_{\text{gesamt}} = -\frac{1}{N} \sum_{j=1}^{N} \sum_{k=1}^{C} y_{j,k} \log(\hat{y}_{j,k}) + \lambda \sum_{i} w_i^2
$$

Hierbei ist:
- $ N $: Die Anzahl der Beispiele im Batch.
- $ C $: Die Anzahl der Klassen.
- $ y_{j,k} $: Das echte Label (one-hot codiert).
- $ \hat{y}_{j,k} $: Die vorhergesagte Wahrscheinlichkeit.
- $ \lambda $: Der Regularisierungsparameter.
- $ w_i $: Die Gewichte des Modells.

In der Methode `training_step` wurde ein L2-Strafterm hinzugefügt. Der Term berechnet die Summe der L2-Normen aller trainierbaren Parameter des Modells. Diese Änderung wurde wie folgt implementiert:

```python
# L2-Regularisierung (Gewichtstrafe)
l2_lambda = 0.01
l2_reg = sum(torch.norm(param, 2) for param in self.model.parameters() if param.requires_grad)
loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1)) + l2_lambda * l2_reg
```

### Bedeutung der Parameter:
- `l2_lambda = 0.01`: Dieser Parameter bestimmt die Stärke der Regularisierung. Ein höherer Wert führt zu einer stärkeren Bestrafung großer Gewichte.
- `torch.norm(param, 2)`: Berechnet die L2-Norm (euklidische Norm) eines Parameters $ param $.
- `sum(...)`: Summiert die L2-Normen aller trainierbaren Parameter.

Die Regularisierung wird zur Kreuzentropieverlustfunktion hinzugefügt, um die Gesamtverluste zu berechnen. Durch die Gewichtregulierung werden die Gewichte des Modells klein gehalten, was folgende Vorteile mit sich bringt:
- **Verbesserte Generalisierung**: Das Modell wird weniger anfällig für Überanpassung.
- **Numerische Stabilität**: Kleinere Gewichte führen zu stabileren Berechnungen.

Die eingeführte L2-Gewichtregulierung ist eine effektive Methode, um die Leistung des Modells auf Testdaten zu verbessern. Sie sorgt dafür, dass die Gewichte klein bleiben und die Generalisierungsfähigkeit erhöht wird, ohne die Modellarchitektur zu verkomplizieren.



In [ ]:
class ReutersLightningModel_l2(pl.LightningModule):
    """
    A PyTorch Lightning implementation of the Keras Sequential model with dropout and weight regularization.
    """

    def __init__(self):
        super(ReutersLightningModel_l2, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(10000, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 46),
            nn.Softmax(dim=1),
        )
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        # Apply L2 regularization (weight decay)
        l2_lambda = 0.01
        l2_reg = sum(
            torch.norm(param, 2)
            for param in self.model.parameters()
            if param.requires_grad
        )
        loss = (
            nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
            + l2_lambda * l2_reg
        )

        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", accuracy, on_epoch=True, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(accuracy.item())
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        predictions = self(data)
        loss = nn.CrossEntropyLoss()(predictions, labels.argmax(dim=1))
        accuracy = (predictions.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("val_accuracy", accuracy, on_epoch=True, prog_bar=True, sync_dist=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(accuracy.item())
        return {"val_loss": loss, "val_accuracy": accuracy}

    def on_validation_epoch_end(self):
        outputs = self.trainer.callback_metrics
        avg_loss = outputs.get("val_loss_epoch", torch.tensor(0.0)).item()
        avg_accuracy = outputs.get("val_accuracy_epoch", torch.tensor(0.0)).item()
        self.val_losses.append(avg_loss)
        self.val_accuracies.append(avg_accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


In [ ]:
# Model instantiation
model4 = ReutersLightningModel_l2()

In [ ]:
# Trainer setup
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1)
trainer.fit(model4, train_loader, validation_loader)

# Plot learning curves
plot_learning_curves(model)


## Weitere Experimente (Hausaufgaben)

- Probieren Sie größere oder kleinere Layer aus: 32 Einheiten, 128 Einheiten usw.
- Sie haben bislang zwei verdeckte Layer verwendet. Probieren Sie nun aus, nur einen einzigen oder drei verdeckte Layer zu verwenden.

## Zusammenfassung

Nehmen Sie Folgendes aus diesem Abschnitt mit:
- Falls Sie versuchen, Datenpunkte N Kategorien zuzuordnen, sollte Ihr NN mit einem Dense-Layer der Größe N enden.
- Bei einer Single-Label-Mehrfachklassifizierungsaufgabe sollte Ihr NN mit einer `softmax`-Aktivierung enden, damit es eine Wahrscheinlichkeitsverteilung der N Klassen ausgibt.
- Bei Aufgaben dieser Art sollten Sie als Verlustfunktion fast immer die kategoriale Kreuzentropie (`categorical_crossentropy`) verwenden. Sie minimiert die Differenz zwischen den Wahrscheinlichkeitsverteilungen der Ausgabe des NNs und der tatsächlichen Verteilung der Zielwerte.
- Zur Handhabung der Klassenbezeichnungen einer Mehrfachklassifizierung stehen zwei Möglichkeiten zur Verfügung:
    - die **kategoriale Codierung** (die auch als **One-hot-Codierung** bezeichnet wird) mit der Verwendung der Verlustfunktion `categorial_crossentropy` 
    - und die die **Codierung der Klassenbezeichnungen als Integer** und die Verwendung der Verlustfunktion `sparse_categorial_crossentropy`
- Wenn Sie die Daten sehr vielen Kategorien zuordnen müssen, sollten Sie Informationslecks durch zu kleine zwischenliegende Layer im NN vermeiden.